In [1]:
import os
import openai
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time


# Connect to ChatGpt Api

In [2]:
openai.organization = "org-RuJeUUCVE9l16B6VH1CFTTJn"
openai.api_key = os.getenv("CHATGPTKEY")
openai.Model.list()

<OpenAIObject list at 0x1208b2c20> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

## Check if the connection and saving is working


In [3]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-78xvaopOaffgKbTnpmZCB4Q1Nvj7J at 0x1208ab220> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The World Series in 2020 was played primarily at Globe Life Field in Arlington, Texas due to the COVID-19 pandemic. However, some games were also played at Dodger Stadium in Los Angeles.",
        "role": "assistant"
      }
    }
  ],
  "created": 1682371342,
  "id": "chatcmpl-78xvaopOaffgKbTnpmZCB4Q1Nvj7J",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 41,
    "prompt_tokens": 57,
    "total_tokens": 98
  }
}

## Start testing

In [4]:
# open our test dataframe from the csv
df = pd.read_csv('data/test/testset.csv')
df.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,priceCurrency_right,specTableContent_right,cluster_id_right,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction
0,1840,98621092,NaN,TAG Heuer Monaco Chronograph Calibre 11 Automatic,Design inspired by the watch worn by Steve McQ...,"£4,195.00",NaN,NaN,556904,2004031,...,GBP,NaN,556904,98621092#2004031,1,False,"[3.7184028625, -4.1772656441]",0,[1.0],1
1,710,47887901,NaN,Apple AirPods (2nd Generation) Bluetooth Earbu...,Staples.com: Apple AirPods MRXJ2AM/A In the Ea...,199,USD,NaN,1994510,49097388,...,EUR,NaN,767259,47887901#49097388,0,True,"[-1.7439045906000001, 1.717010498]",1,[0.0],0
2,2065,60950283,NaN,Fujifilm Instax Square Instant Film - 10 Photos,The Fujifilm Instax Square Instant Film is for...,11.99,GBP,NaN,1457721,25935656,...,NaN,NaN,1118780,60950283#25935656,0,True,"[-2.1758391857, 2.2021055222]",1,[0.0],0
3,3869,435472,NaN,Evans EMAD Clear Bass Drum Head 20 Inch,"20"" drum head made using a single ply of 10mil...",46.06,USD,NaN,800940,55131100,...,CAD,NaN,800940,435472#55131100,1,False,"[3.2268047333, -3.6520571709]",0,[1.0],1
4,1787,88641225,NaN,"8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...","8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...",74.99,CAD,NaN,288081,86250508,...,EUR,NaN,836466,88641225#86250508,0,True,"[-2.2421159744, 2.2641017437]",1,[0.0],0


In [5]:
# create a new column for the chatbot response
df['chatbot_response'] = ''
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [6]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = [
        {"role": "system", "content": "You are a helpful assistant that tries to tell if two products are the same. Two products are only the same if all attributes match! Only answer yes or no."},
        {'role': 'user', 'content': 'product 1: Corsair Vengeance LPX RAM 16GB 2400MHz DDR4 UDIMM 288 Pin Desktop Memory, product 2: 16GB Corsair Vengeance LPX schwarz DDR4-2400 DIMM CL14 Single'},
        {'role': 'assistant', 'content': 'No.'},
        {'role': 'user', 'content': 'Thats incorrect, they are the same!'},
        {"role": "user",
            "content": f"product 1: {row['title_left']}, product 2: {row['title_right']}"},
    ]
    error = ""
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=2,
            )
            # if the above call succeeds, break out of the loop and continue
            break
        except Exception as e:
            error = e
            print(f"Error: {e}")
            # if the call fails, wait for one minute before retrying
            time.sleep(60)

    # check if the response was set successfully
    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_response'] = response['choices'][0]['message']['content']
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_response'] = -1
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
df.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
0,1840,98621092,NaN,TAG Heuer Monaco Chronograph Calibre 11 Automatic,Design inspired by the watch worn by Steve McQ...,"£4,195.00",NaN,NaN,556904,2004031,...,98621092#2004031,1,False,"[3.7184028625, -4.1772656441]",0,[1.0],1,Yes.,{'id': 'chatcmpl-78xvcLCCLpkYHamXjCVLXDXpDCW0h...,"[{'role': 'system', 'content': 'You are a help..."
1,710,47887901,NaN,Apple AirPods (2nd Generation) Bluetooth Earbu...,Staples.com: Apple AirPods MRXJ2AM/A In the Ea...,199,USD,NaN,1994510,49097388,...,47887901#49097388,0,True,"[-1.7439045906000001, 1.717010498]",1,[0.0],0,No.,{'id': 'chatcmpl-78xvd6TM7gUrm4qA5RtdwD4lCtB4s...,"[{'role': 'system', 'content': 'You are a help..."
2,2065,60950283,NaN,Fujifilm Instax Square Instant Film - 10 Photos,The Fujifilm Instax Square Instant Film is for...,11.99,GBP,NaN,1457721,25935656,...,60950283#25935656,0,True,"[-2.1758391857, 2.2021055222]",1,[0.0],0,Yes.,{'id': 'chatcmpl-78xvdkWzXcRIr5N1AO3vvHPr5bbAV...,"[{'role': 'system', 'content': 'You are a help..."
3,3869,435472,NaN,Evans EMAD Clear Bass Drum Head 20 Inch,"20"" drum head made using a single ply of 10mil...",46.06,USD,NaN,800940,55131100,...,435472#55131100,1,False,"[3.2268047333, -3.6520571709]",0,[1.0],1,"Yes,",{'id': 'chatcmpl-78xveY1r26ksJpZcyfnyX4vK5q413...,"[{'role': 'system', 'content': 'You are a help..."
4,1787,88641225,NaN,"8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...","8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...",74.99,CAD,NaN,288081,86250508,...,88641225#86250508,0,True,"[-2.2421159744, 2.2641017437]",1,[0.0],0,No.,{'id': 'chatcmpl-78xvekYkJS1oZqabJ2f0Bq01NmtkQ...,"[{'role': 'system', 'content': 'You are a help..."


In [9]:
# get the number of rows were the chatbot respionse is not empty
df[df['chatbot_response'] != ''].shape

(100, 27)

In [10]:
df.tail()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
95,2820,31581824,SHIMANO,SPD-SL SH 11 - tacchette,The Shimano SM-SH11 cleats serve as a connecti...,15.33,EUR,NaN,532781,75286119,...,31581824#75286119,0,True,"[-4.3649616241, 4.6570587158]",1,[0.0],0,No.,{'id': 'chatcmpl-78xwLNWhYoFaPw3V0vEUSLTu8U4MZ...,"[{'role': 'system', 'content': 'You are a help..."
96,4223,6002122,NaN,LOGITECH - STEREO HEADSET H151 ANALOG - EMEAIN...,LOGITECH STEREO HEADSET H151 ANALOG - EMEAIN A...,NaN,NaN,NaN,122031,31273059,...,6002122#31273059,0,True,"[-4.3757534027, 4.6587324142]",1,[0.0],0,Yes.,{'id': 'chatcmpl-78xwLyDCctz2bCKi91MVEQWDWtTbG...,"[{'role': 'system', 'content': 'You are a help..."
97,2073,393407,Audemars Piguet,Audemars Piguet Royal Oak Chronograph,Audemars Piguet Royal Oak,36560,USD,NaN,607611,98604797,...,393407#98604797,0,True,"[-4.1834125519, 4.471534729]",1,[0.0],0,Yes.,{'id': 'chatcmpl-78xwMfVe1R00rUTtoDQdAvqvg98pB...,"[{'role': 'system', 'content': 'You are a help..."
98,3917,89292552,NaN,Shimano SPD SL Cleat Set,Shimano cleats come in 3 different colours the...,39.95,AUD,NaN,431550,75045785,...,89292552#75045785,0,True,"[-3.1417820454, 3.3016252518]",1,[0.0],0,Yes.,{'id': 'chatcmpl-78xwMqij6SJ2RkoZ1Sb4FFMsWDuBl...,"[{'role': 'system', 'content': 'You are a help..."
99,1910,10830960,NaN,SANDISK EXTREME SDXC 128GB150/70 MB/s V30 UHS-...,NaN,151.98,PLN,NaN,2427670,6664354,...,10830960#6664354,0,True,"[-1.4274802208000001, 1.4668471813]",1,[0.0],0,No.,{'id': 'chatcmpl-78xwNTejKzxI7clf4UcPYWKsGjNKC...,"[{'role': 'system', 'content': 'You are a help..."


In [11]:
def clean_response(response):
    if "yes" in response.lower():
        return 1
    elif "no" in response.lower():
        return 0
    else:
        return -1

In [23]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response'].apply(lambda x: clean_response(x))

In [7]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/chat_gpt/{now}_simple_promt.json')

In [13]:
df.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,1840,98621092,NaN,TAG Heuer Monaco Chronograph Calibre 11 Automatic,Design inspired by the watch worn by Steve McQ...,"£4,195.00",NaN,NaN,556904,2004031,...,1,False,"[3.7184028625, -4.1772656441]",0,[1.0],1,Yes.,{'id': 'chatcmpl-78xvcLCCLpkYHamXjCVLXDXpDCW0h...,"[{'role': 'system', 'content': 'You are a help...",1
1,710,47887901,NaN,Apple AirPods (2nd Generation) Bluetooth Earbu...,Staples.com: Apple AirPods MRXJ2AM/A In the Ea...,199,USD,NaN,1994510,49097388,...,0,True,"[-1.7439045906000001, 1.717010498]",1,[0.0],0,No.,{'id': 'chatcmpl-78xvd6TM7gUrm4qA5RtdwD4lCtB4s...,"[{'role': 'system', 'content': 'You are a help...",0
2,2065,60950283,NaN,Fujifilm Instax Square Instant Film - 10 Photos,The Fujifilm Instax Square Instant Film is for...,11.99,GBP,NaN,1457721,25935656,...,0,True,"[-2.1758391857, 2.2021055222]",1,[0.0],0,Yes.,{'id': 'chatcmpl-78xvdkWzXcRIr5N1AO3vvHPr5bbAV...,"[{'role': 'system', 'content': 'You are a help...",1
3,3869,435472,NaN,Evans EMAD Clear Bass Drum Head 20 Inch,"20"" drum head made using a single ply of 10mil...",46.06,USD,NaN,800940,55131100,...,1,False,"[3.2268047333, -3.6520571709]",0,[1.0],1,"Yes,",{'id': 'chatcmpl-78xveY1r26ksJpZcyfnyX4vK5q413...,"[{'role': 'system', 'content': 'You are a help...",1
4,1787,88641225,NaN,"8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...","8GB (2x4GB) HyperX Fury Blue DDR3, 1600MHz, CL...",74.99,CAD,NaN,288081,86250508,...,0,True,"[-2.2421159744, 2.2641017437]",1,[0.0],0,No.,{'id': 'chatcmpl-78xvekYkJS1oZqabJ2f0Bq01NmtkQ...,"[{'role': 'system', 'content': 'You are a help...",0


In [14]:
# Check the accuracy of the chatbot 
df[df['chatbot_response_clean'] == df['label']].shape[0]
print(f"Accuracy: {df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]}")

Accuracy: 0.49


In [15]:
# Check the number of -1 responses
df[df['chatbot_response_clean'] == -1].shape[0]

0

In [19]:
# Look at 5 complete chatbot responses dont limit the output length
# set the max_rows and max_columns options to None
# set the max_colwidth option to None
pd.set_option('display.max_colwidth', 300)

# display the first 5 chatbot responses where chatbot_response_raw is not -1
df['chatbot_response_raw'].head(3)

0    {'id': 'chatcmpl-78r6n5Gf7efpLf6pZOBMB3t7aFpJr', 'object': 'chat.completion', 'created': 1682345129, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 102, 'completion_tokens': 2, 'total_tokens': 104}, 'choices': [{'message': {'role': 'assistant', 'content': 'No.'}, 'finish_reason': 'len...
1    {'id': 'chatcmpl-78r6n3fiY9HHSzbVUdSjYlaHIxuM6', 'object': 'chat.completion', 'created': 1682345129, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 85, 'completion_tokens': 2, 'total_tokens': 87}, 'choices': [{'message': {'role': 'assistant', 'content': 'The two'}, 'finish_reason': 'l...
2    {'id': 'chatcmpl-78r6ogVr4MiIfqGXZuFmrWmX1li6T', 'object': 'chat.completion', 'created': 1682345130, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 110, 'completion_tokens': 2, 'total_tokens': 112}, 'choices': [{'message': {'role': 'assistant', 'content': 'No.'}, 'finish_reason': 'len...
Name: chatbot_response_raw, dtype: object

In [25]:
# Lets look at all mistakes only show the title_left, title_right, label and chatbot_response_clean
df[df['chatbot_response_clean'] != df['label']][['title_left', 'title_right', 'label', 'chatbot_response_clean']]


,title_left,title_right,label,chatbot_response_clean
2,Fujifilm Instax Square Instant Film - 10 Photos,Fujifilm Instax Square Film Black Frame 10 Shots,0,1
6,Memoria DDR3 1600Mhz Kit 8GB HyperX Kingston (2x4GB),HyperX Fury DDR3 1600MHz 8GB,0,1
7,"Kingston Canvas Select UHS-I, 64 GB",Memoire SDXC KINGSTON 64GB Canvas Select microSDHC (microSD) + Adapter SD,1,0
12,Corsair Air ML140 PRO MAGNETIC 140mm LED blue 4-pin,Corsair ML Series ML140 PRO LED Premium Magnetic Levitation,0,1
13,Сървър HEWLETT PACKARD HP 900GB 6G SAS 10K rpm SFF (2.5-inch) SC Enterprise 3yr Warranty Hard Drive,"HP 900GB 6G SAS 10K 2.5\"" SC Server Hard Disk",0,1
14,TP-Link EAP225-Outdoor AC1200 Dual Band Outdoor Access Point,ACCESS POINT TP-LINK EAP EAP225 OUTDOOR - TPN0125,0,1
16,TAG Heuer Automatic Watch WAZ2011.BA0842,Formula 1 Calibre 5 Automatic Watch,1,0
17,Canon EF 100mm F2.8L Macro IS USM + BONUS,EF 100mm f/2.8L Macro IS USM,0,1
18,Fujifilm Instax Square Film - 20 Sheets,Fujifilm Instax Square Instant Film - 10 Photos,1,0
22,3M MOBILE INTERACTIVE SOLUTION PF220W9F LIGHTWEIGHT FRAMED PRIVACY FILTER,"3M - Privacy Filter 19\""\"" WideS",1,0
